# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [87]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('assets/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [88]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [89]:
def answer_one():
    from collections import Counter
    # YOUR CODE HERE
    #total data
    total = spam_data.shape[0]
    count=Counter(spam_data['target'])
    pos = count[1]
    
    return 100*(pos/total)

answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [90]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():

    # YOUR CODE HERE
    vect = CountVectorizer().fit(X_train)
    fea_vect = vect.get_feature_names()
    length_features = [len(x) for x in fea_vect]
    longest_fea = fea_vect[np.argmax(length_features)]

   
    
    
    
   
    return longest_fea

answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [91]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():

    # YOUR CODE HERE
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    predictions = model.predict(vect.transform(X_test))
    auc_score = roc_auc_score(y_test, predictions)
    return auc_score

answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters. The transformed data will be a compressed sparse row matrix where the number of rows is the number of documents in `X_train`, the number of columns is the number of features found by the vectorizer in each document, and each value in the sparse matrix is the tf-idf value. First find the **max** tf-idf value for every feature.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

Put these features in two series where each series is sorted by tf-idf value. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. Any entries with identical tf-ids should appear in lexigraphically increasing order by their feature name in boh series. For example, if the features "a", "b", "c" had the tf-idfs 1.0, 0.5, 1.0 in the series with the largest tf-idfs, then they should occur in the returned result in the order "a", "c", "b" with values 1.0, 1.0, 0.5.

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():

    # YOUR CODE HERE
    vect = TfidfVectorizer()
    
    # get tf_idf values
    tf_df_values = vect.fit_transform(X_train)
    
    #get features/words
    feature_names = np.array(vect.get_feature_names())
    
    
    tf_df_max = tf_df_values.max(0).toarray()[0]
    
    
    #get the indices/words values (arranged in order)
    sorted_tf_idxs = tf_df_max.argsort()
    
    #get the tf idfs (arranged in order)
    sorted_tf_idfs = tf_df_max[sorted_tf_idxs]
    
    
    smallest = pd.Series(sorted_tf_idfs[:20], index=feature_names[sorted_tf_idxs[:20]])                    
    largest = pd.Series(sorted_tf_idfs[-20:][::-1], index=feature_names[sorted_tf_idxs[-20:][::-1]])
    

    return (smallest, largest)

answer_four()

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 dt

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [103]:
def answer_five():

    # YOUR CODE HERE
    vect = TfidfVectorizer(min_df = 3)
    X_train_vectorized = vect.fit_transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)
    
    predictions = model.predict_proba(X_test_vectorized)[:, 1]
    
    auc_score = roc_auc_score(y_test, predictions)
    
    return auc_score

answer_five()

0.9954968337775665

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [94]:
def answer_six():

    # YOUR CODE HERE
    #create an additional column with the legth of each text
    spam_data['length'] = spam_data['text'].apply(lambda x:len(x))
    
    #get the means for target ==0 and ==1 
    mean_notspam = np.mean(spam_data['length'][spam_data['target'] == 0])
    mean_spam = np.mean(spam_data['length'][spam_data['target'] == 1])                       
    return (mean_notspam, mean_spam)

answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [95]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*Hint: Since probability is set to false, use the model's `decision_function` on the test data when calculating the target scores to use in roc_auc_score*

*This function should return the AUC score as a float.*

In [102]:
from sklearn.svm import SVC

def answer_seven():

    # YOUR CODE HERE
    #  new feature to add - document length adding it
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    
    vect = TfidfVectorizer(min_df = 5)
    X_train_vectorized = vect.fit_transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    #add new feature to doc-term matrix
    X_train_vectorized = add_feature(X_train_vectorized, len_train)
    X_test_vectorized = add_feature(X_test_vectorized, len_test)
    
    #model SVM with C=10000 (low regularizations since C is large --> possible overfitting)
    model = SVC(C=10000)
    model.fit(X_train_vectorized, y_train)
    predictions = model.decision_function(X_test_vectorized)
    auc_score = roc_auc_score(y_test, predictions)
    
    return auc_score

answer_seven()

0.9963202213809143

### Question 8

What is the average number of digits per document for not spam and spam documents?

*Hint: Use `\d` for digit class*

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [97]:
def answer_eight():

    # YOUR CODE HERE
    ##Initial code that did not work
    #dig_spam = [sum(char.isnumeric() for char in x) for x in spam_data.loc[spam_data['target']==1,'text']]
    #dig_notspam = [sum(char.isnumeric() for char in x) for x in spam_data.loc[spam_data['target']==0,'text']]
    #mean_spam = np.mean(dig_spam)
    #mean_notspam = np.mean(dig_notspam)
    
    #version 2 gave the same result
    import re
    #add column digit count
    m = spam_data.shape[0]
    for n in range(m):       
        digit_count_m = len(re.findall('\d', spam_data.loc[n,'text']))
        spam_data.loc[n,'digit_count'] = digit_count_m
    
    #get the means for target ==0 and ==1 
    mean_notspam = np.mean(spam_data['digit_count'][spam_data['target'] == 0])
    mean_spam = np.mean(spam_data['digit_count'][spam_data['target'] == 1])                       
    return (mean_notspam, mean_spam)


answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100` and `max_iter=1000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [107]:
from sklearn.linear_model import LogisticRegression

def answer_nine():

    # YOUR CODE HERE
    #getting the new features length of documents and number od digits per document
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    dig_train = X_train.str.count('\d')
    dig_test = X_test.str.count('\d')
    
    
    vect =  TfidfVectorizer(min_df=5, ngram_range=(1,3))
    X_train_vectorized = vect.fit_transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    #add new feature to doc-term matrix
    X_train_vectorized = add_feature(X_train_vectorized, [len_train, dig_train])
    X_test_vectorized = add_feature(X_test_vectorized, [len_test, dig_test])
   
    
    #classifier model and predictions
    model = LogisticRegression(C=100, max_iter=1000)
    model.fit(X_train_vectorized, y_train)
    predictions = model.predict_proba(X_test_vectorized)[:, 1]
    
    auc_score = roc_auc_score(y_test, predictions)
    
    return auc_score

answer_nine()

0.9973218681561211

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [99]:
def answer_ten():

    # YOUR CODE HERE
    nochar_spam = spam_data.loc[spam_data['target']==1,'text'].str.count('\W')
    nochar_notspam = spam_data.loc[spam_data['target']==0,'text'].str.count('\W')
    mean_spam = np.mean(nochar_spam)
    mean_notspam = np.mean(nochar_notspam)
    
    return (mean_notspam, mean_spam)

answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the **first 2000 rows** of training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100 and max_iter=1000. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [105]:
def answer_eleven():

    # YOUR CODE HERE
    # reduce the train data set
    X_train_reduced = X_train.iloc[:2000]
    y_train_reduced = y_train.iloc[:2000]
    
    
    #getting the new features length of documents, number od digits per document and number of non-word characters per document
    len_train = [len(x) for x in X_train_reduced]
    len_test = [len(x) for x in X_test]
    dig_train = X_train_reduced.str.findall(r'(\d)').str.len()
    dig_test = X_test.str.findall(r'(\d)').str.len()
    nonchar_train = X_train_reduced.str.findall(r'(\W)').str.len()
    nonchar_test = X_test.str.findall(r'(\W)').str.len()
    
    #vectorization fit and transform following the conditions presented
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train_reduced)
    X_train_vectorized = vect.transform(X_train_reduced)
    X_test_vectorized = vect.transform(X_test)
    
    #add new feature to doc-term matrix
    X_train_vectorized = add_feature(X_train_vectorized, [len_train, dig_train, nonchar_train])
    X_test_vectorized = add_feature(X_test_vectorized, [len_test, dig_test, nonchar_test])
    
    #classifier model and predictions
    model = LogisticRegression(C=100, max_iter=1000)
    model.fit(X_train_vectorized, y_train_reduced)
    predictions = model.predict_proba(X_test_vectorized)[:, 1]
    auc_score = roc_auc_score(y_test, predictions)
   

    #calculating the coeficients
    feature_names = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    sorted_coef_index = model.coef_[0].argsort()
    small_coef = list(feature_names[sorted_coef_index[:10]])
    larg_coef = list(feature_names[sorted_coef_index[:-11:-1]])


    
    return (auc_score, small_coef, larg_coef)

answer_eleven()

(0.997568035583926,
 ['n ', ' i', 'at', 'he', ' m', '..', 'us', 'go', ' lo', ' bu'],
 ['digit_count', 'ne', ' st', 'co', 's ', 'xt', 'lt', 'xt ', ' ne', 'der'])